## Accessing PAIRS Image Data through API 


Before importing api_wrapper, had to install some dependencies: 
- pip install geopandas, geojson, coloredlogs
- conda install gdal

In [1]:
import sys
import os
# save present working directory so we can switch back to it 
pwd = os.getcwd()
# change directory to get access to api_wrapper
os.chdir('../scalable-landcover-classification/Querying')
import api_wrapper as paw
# switch back to original directory
os.chdir(pwd)
import pandas as pd
import numpy as np
import geojson
import urllib
from datetime import datetime
from pytz import UTC
import time
import random
import logging
import requests
from requests.auth import HTTPBasicAuth
import os, zipfile, io
from osgeo import gdal
import PIL

Config

In [2]:
PAIRS_USER = "ams792@cornell.edu"
PAIRS_PASS = "myibmpairspassword"
server = "https://pairs.res.ibm.com"
auth = (PAIRS_USER, PAIRS_PASS)
path = './downloads' #Download path

In [48]:
lat1 = 41.0100756423
lat2 = 41.0338409682
lon1 = -73.7792749922
lon2 = -73.7582464736

In [53]:
response = requests.post(
    json = {
    "layers" : [
        {
            "id" : 35445
        },
    ],
    "spatial" : {
        "type" : "square", "coordinates" : [lat1, lon1, lat2, lon2], # [lat_min, lon_min, lat_max, lon_max] 
    },
    "temporal" : {
        "intervals" : [
            {
                "start":    '2018-09-20T0:0:0',
                "end" :     '2018-09-20T0:0:0',
            }
        ]

    }
    },
    url=f'{server}/v2/query',
    auth=auth,
)

In [54]:
res = response.json()
id = res['id']
print("ID: ",id)

ID:  1539446400_07146842


Check status of query, make sure it's finished before downloading

In [56]:
response = requests.get(
    url=f'{server}/v2/queryjobs/{id}',
    auth=auth,
)
response.json()

{'exPercent': 0.0,
 'flag': False,
 'folder': '1539453546841',
 'hadoopId': None,
 'id': '1539446400_07146842',
 'neLat': 41.0338402,
 'neLon': -73.7582474,
 'nickname': None,
 'pdStatus': None,
 'pql': None,
 'ready': False,
 'rtStatus': 'Failed',
 'start': 1539453546842,
 'status': 'Failed',
 'statusCode': 40,
 'swLat': 41.0100746,
 'swLon': -73.779274}

Download and extract to files

In [11]:
download = requests.get(
    f'{server}/download/{id}', auth=auth, stream=True,
)

z = zipfile.ZipFile(io.BytesIO(download.content))

## Extract to ./downloads
z.extractall(path)

Convert to np array

In [12]:
## Takes file name and converts to np array
def tiff2array(fn):
    ## Special gdal dataset
    dataset = gdal.Open(fn)
    array = np.array(dataset.GetRasterBand(1).ReadAsArray())
    return array

## Will go through filenames and put the image arrays in here
images_dict = {}

## Loop through files in downloads directory (if multiple)
for i, filename in enumerate(os.listdir(path)):
    if filename.endswith(".tiff"): 
        path_to_file = path + '/' + filename
        images_dict[i] = tiff2array(path_to_file)

## Note: keys start at 1 for image_dict



In [17]:
images_dict[1].shape

(32, 64)

In [18]:
images_dict


{1: array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
       dtype=float32)}